In [ ]:
# 导入Py_Service
import asyncio
import websockets
import nest_asyncio
import sys

nest_asyncio.apply()


async def listen_for_connection(uri, timeout):
    start_time = asyncio.get_event_loop().time()
    while True:
        try:
            async with websockets.connect(uri) as websocket:
                print("发现服务")
                return
        except (ConnectionRefusedError, OSError):
            if (asyncio.get_event_loop().time() - start_time) > timeout:
                print("未能在指定时间内发现指定端口的 websocket 服务")
                sys.exit(1)
            else:
                await asyncio.sleep(1)


async def is_websocket_service_available(uri):
    try:
        async with websockets.connect(uri) as websocket:
            return True
    except (ConnectionRefusedError, OSError):
        return False


async def waitForSecond(sec: int):
    await asyncio.sleep(sec)


sys.path.append("/Users/nobody/Documents/develop/GitHub/Services/PY_Service")
from utils import pyServiceUtils

from BB.app.services.BBTs.BBTs_V8_Debugger import BBTs_V8_Debugger

_v8_Debugger: BBTs_V8_Debugger = pyServiceUtils.getSubSvrByName("BB", "BBTs", "BBTs_V8_Debugger")
_uri = "ws://localhost:8080"

In [ ]:
'''
http://127.0.0.1:8888/notebooks/JupyterLab/python/BB_TS_SLG/Debug/QuickQueue_Debug.ipynb
'''

_v8_Debugger.clearLogs()  # 清理日志

_V8_is_online = await is_websocket_service_available(_uri)
if _V8_is_online:
    _v8_Debugger.reset()  # 重置环境

await listen_for_connection(_uri, 20)  # 等待 Unity 中 V8 启动

await waitForSecond(3)  # 等主界面启动

_v8_Debugger.debugModule("QuickQueue")  #